# Data Preparation

In [ ]:
import numpy as np
import pandas as pd
import pyhdb
from sqlalchemy import create_engine
from sqlalchemy.types import NVARCHAR
%matplotlib inline

In [ ]:
chunksize = 100000
churn = pd.read_csv('data_20171001_20190930.csv', chunksize=100000, iterator=True)
data = pd.concat(churn, ignore_index=True)

In [ ]:
data['date'] = pd.to_datetime(data.date, format="%Y%m%d")
from lifetimes.utils import summary_data_from_transaction_data
# lifetimes provides a transaction log -> rfm util function
training = summary_data_from_transaction_data(
     data,
    'customer_id',
    'date',
    monetary_value_col = 'amount',
    observation_period_end=pd.to_datetime('2018-09-30'),
    freq='D'
)
training.to_csv('training_day.csv')

In [ ]:
# Reading the Training RFM(20171001~20180930) anad Test RFM(20181001~20190930)
training = pd.read_csv("training_day.csv",index_col='customer_id')
test = pd.read_csv("test_day.csv",index_col='customer_id')

# 1. Predicting the customer future transactions

### 1.1 Delete the outliers

In [ ]:
# Looking the rfm frequency distribution
%matplotlib inline
import matplotlib.pyplot as plt
training['frequency'].plot(kind='hist', bins=50)
plt.ylabel('Number of Customers (N=1.52million)')
plt.xlabel('Numbers of repeated daily shopping')
plt.title('Numbers of repeated daily shopping by NWNZ clubcard customers from 2017-10-01 to 2018-09-30')

In [ ]:
# Looking the rfm frequency distribution
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
plt.xlim([0,1000])

ax = training['monetary_value'].plot(kind='hist', bins=1000)

formatter = ticker.FormatStrFormatter('$%0.1f') #declaring the formatter with the $ sign and y_values with 1 decimalplace
ax.xaxis.set_major_formatter(formatter)

plt.ylabel('Number of Customers (N=1.52million)')
plt.xlabel('Average daily purchased spent')
plt.title('Average daily purchased spent by NWNZ clubcard customers from 2017-10-01 to 2018-09-30')

In [ ]:
# Looking the frequency between 150 <
import matplotlib.pyplot as plt
training[(training['frequency'] >= 141) &(training['frequency'] <= 361)]['frequency'].plot(kind='hist', bins=50)
plt.ylabel('Number of Customers (N=1.52million)')
plt.xlabel('Numbers of repeated daily shopping')
plt.title('Numbers of repeated daily shopping by NWNZ clubcard customers from 2017-10-01 to 2018-09-30')

In [ ]:
# Looking the monetary value $155<
import matplotlib.pyplot as plt
ax = training[(training['monetary_value'] >= 155) &(training['monetary_value'] <= 1000)]['monetary_value'].plot(kind='hist', bins=50)
formatter = ticker.FormatStrFormatter('$%0.1f') #declaring the formatter with the $ sign and y_values with 1 decimalplace
ax.xaxis.set_major_formatter(formatter)

plt.ylabel('Number of Customers (N=1.52million)')
plt.xlabel('Average daily purchased spent')
plt.title('Average daily purchased spent by NWNZ clubcard customers from 2017-10-01 to 2018-09-30')

In [ ]:
training_describe = training.describe()[['frequency','monetary_value']]
training_describe.columns = ['Numbers of repeated daily shopping','Average daily purchased spent']
pd.options.display.float_format = '{:.5f}'.format
training_describe['Average daily purchased spent'] = training_describe['Average daily purchased spent'].map('${:,.2f}'.format)
training_describe

In [ ]:
# Delete the Outliers
training = training.loc[(training['frequency'] > 0) & (training['frequency'] <= 250)]

### 1.2 Model

In [ ]:
# Using the BG/NBD model
from lifetimes import BetaGeoFitter
bgf = BetaGeoFitter(penalizer_coef=0.05)
bgf.fit(training['frequency'], training['recency'], training['T'])
bgf.summary

### 1.3 Customer Nuber of Repeated Daily Purchased prediction for next 1 year

In [ ]:
#Predict for next 1 years = 365 days
training = training[training['frequency']>0]
t = 365
training['predicted_purchases'] = bgf.conditional_expected_number_of_purchases_up_to_time(t, training['frequency'], training['recency'], training['T'])
training.head(5)

### 1.4 Calibration dataset VS holdout dataset

In [ ]:
# Reading the Training RFM(20171001~20180930) anad Test RFM(20181001~20190930)
training = pd.read_csv("training_day.csv",index_col='customer_id')
test = pd.read_csv("test_day.csv",index_col='customer_id')

# Creating the Calibration_Holdout dataset
training.columns = ['frequency_cal','recency_cal','T_cal','monetary_value_cal']
test.columns = ['Actual Repeated Daily Purchases','recency_holdout','duration_holdout','monetary_value_holdout']
summary_cal_holdout = pd.concat([training[['frequency_cal','recency_cal','T_cal']],test[['Actual Repeated Daily Purchases','duration_holdout']]], axis =1).dropna()

In [ ]:
summary_cal_holdout.head()

In [ ]:
summary_cal_holdout.describe()

In [ ]:
#Delete the outliers
summary_cal_holdout = summary_cal_holdout.query('frequency_cal <= 250')

In [ ]:
# Draw the calibration_purchases_vs_holdout_purchases Graph
def plot_calibration_purchases_vs_holdout_purchases(
    model, calibration_holdout_matrix, kind="frequency_cal", n=7, **kwargs
):
    """
    Plot calibration purchases vs holdout.
    This currently relies too much on the lifetimes.util calibration_and_holdout_data function.
    Parameters
    ----------
    model: lifetimes model
        A fitted lifetimes model.
    calibration_holdout_matrix: pandas DataFrame
        DataFrame from calibration_and_holdout_data function.
    kind: str, optional
        x-axis :"frequency_cal". Purchases in calibration period,
                 "recency_cal". Age of customer at last purchase,
                 "T_cal". Age of customer at the end of calibration period,
                 "time_since_last_purchase". Time since user made last purchase
    n: int, optional
        Number of ticks on the x axis
    Returns
    -------
    axes: matplotlib.AxesSubplot
    """
    from matplotlib import pyplot as plt

    x_labels = {
        "frequency_cal": "Repeated Daily Purchases from 20171001 to 20180930",
        "recency_cal": "Age of customer at last purchase",
        "T_cal": "Age of customer at the end of calibration period",
        "time_since_last_purchase": "Time since user made last purchase",
    }
    summary = calibration_holdout_matrix.copy()
    duration_holdout = summary.iloc[0]["duration_holdout"]

    summary["Prediction Repeated Daily Purchases"] = model.conditional_expected_number_of_purchases_up_to_time(
            duration_holdout, summary["frequency_cal"], summary["recency_cal"], summary["T_cal"])

    if kind == "time_since_last_purchase":
        summary["time_since_last_purchase"] = summary["T_cal"] - summary["recency_cal"]
        ax = (
            summary.groupby(["time_since_last_purchase"])[["Actual Repeated Daily Purchases", "Prediction Repeated Daily Purchases"]]
            .mean()
            .iloc[:n]
            .plot(**kwargs)
        )
    else:
        ax = summary.groupby(kind)[["Actual Repeated Daily Purchases", "Prediction Repeated Daily Purchases"]].mean().iloc[:n].plot(**kwargs)

    plt.title("Actual Repeated Daily Purchases vs Prediction Repeated Daily Purchases")
    plt.xlabel(x_labels[kind])
    plt.ylabel("Average of Repeated Daily Purchases from 20181001 to 20190930")
    plt.legend()

    return ax

import matplotlib.pyplot as plt
bgf = BetaGeoFitter(penalizer_coef=0.05)
bgf.fit(summary_cal_holdout['frequency_cal'], summary_cal_holdout['recency_cal'], summary_cal_holdout['T_cal'])
fig = plt.figure(figsize=(16,12))
plot_calibration_purchases_vs_holdout_purchases(bgf, summary_cal_holdout, n=250)

### 1.5 Prediction Error

In [ ]:
summary_cal_holdout = summary_cal_holdout[summary_cal_holdout['frequency_cal']>0]
t = 365
summary_cal_holdout['predicted_purchases'] = bgf.conditional_expected_number_of_purchases_up_to_time(t, summary_cal_holdout['frequency_cal'], summary_cal_holdout['recency_cal'], summary_cal_holdout['T_cal'])
holdout = summary_cal_holdout['Actual Repeated Daily Purchases'].sum(axis = 0)
prediction = summary_cal_holdout['predicted_purchases'].sum(axis = 0)
(prediction-holdout)/holdout

# 2. Predicting the customer probability of Churn

### 2.1 Calculating the probability of Churn at the 20190930

In [ ]:
# Import the Training and test dataset
training = pd.read_csv("training_day.csv",index_col='customer_id')
test = pd.read_csv("test_day.csv",index_col='customer_id')

#Delete outliers
training = training.loc[(training['frequency'] > 0) & (training['frequency'] <= 250)]

# Using the BG/NBD model
from lifetimes import BetaGeoFitter
bgf = BetaGeoFitter(penalizer_coef=0.05)
bgf.fit(training['frequency'], training['recency'], training['T'])
bgf.summary

In [ ]:
#Predict that customers still alive on the date of 2019-09-30
test_probability_alive = test[test['frequency']>0]
test_probability_alive['Churn'] = 1 - bgf.conditional_probability_alive(test_probability_alive['frequency'], test_probability_alive['recency'], test_probability_alive['T'])
test_probability_alive.head(5)

In [ ]:
# Distribution for the pobability of alive 
%matplotlib inline
import matplotlib.pyplot as plt
test_probability_alive['Churn'].plot(kind='hist', bins=50)
plt.ylabel('Number of Customers')
plt.xlabel('Probability of Churn')
plt.title('Customer Probability of Churn Distributaion')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle

data = test_probability_alive['Churn']

N, bins, patches = plt.hist(data, 30, ec="k")

cmap = plt.get_cmap('jet')
low = cmap(0.5)
medium =cmap(0.25)
high = cmap(0.8)
very_high = cmap(1.0)

for i in range(0,1):
    patches[i].set_facecolor(low)
for i in range(1,6):
    patches[i].set_facecolor(medium)
for i in range(6,15):
    patches[i].set_facecolor(high)
for i in range(15,30):
    patches[i].set_facecolor(very_high)

#create legend
handles = [Rectangle((0,0),1,1,color=c,ec="k") for c in [low,medium, high,very_high]]
labels= ["safe","Waving", "Accelerated churn", "Churned"]
plt.legend(handles, labels)

plt.xlabel("Probability of Churn", fontsize=10)  
plt.ylabel("Number of Customers", fontsize=10)
plt.title('Customer Probability of Churn Distributaion', fontsize=16)
plt.xticks(fontsize=14)  
plt.yticks(fontsize=14)

plt.gca().spines["top"].set_visible(False)  
plt.gca().spines["right"].set_visible(False)

plt.show()

In [ ]:
# Portion of waving customers
test_probability_alive[(test_probability_alive['Churn'] > 0.05) & ((test_probability_alive['Churn'] <=0.2))].shape[0]/test_probability_alive.shape[0]

### 2.2 Probability of Churned by Loyalty Group

In [ ]:
RFM_Customer = pd.read_csv('CHURN_RFM_CUSTOMER.csv')
bb = test_probability_alive.reset_index('customer_id')[['customer_id','Churn']]
RFM_Customer_Alive = pd.merge(bb, RFM_Customer, how='inner', left_on = 'customer_id', right_on = 'BUSINESS_PARTNER_ID')
RFM_Customer_Alive.head()

#### VIP

In [ ]:
RFM_Customer_Alive_VIP = RFM_Customer_Alive[RFM_Customer_Alive['LOYALTY_GROUP_MD'] == 'VIP']
safe = RFM_Customer_Alive_VIP[(RFM_Customer_Alive_VIP['Churn'] >=0) & (RFM_Customer_Alive_VIP['Churn'] <= 0.05)].count()/RFM_Customer_Alive_VIP.count()
Waving = RFM_Customer_Alive_VIP[(RFM_Customer_Alive_VIP['Churn'] >0.05) & (RFM_Customer_Alive_VIP['Churn'] <= 0.2)].count()/RFM_Customer_Alive_VIP.count()
Accelerated_Churn = RFM_Customer_Alive_VIP[(RFM_Customer_Alive_VIP['Churn'] >0.2) & (RFM_Customer_Alive_VIP['Churn'] <= 0.5)].count()/RFM_Customer_Alive_VIP.count()
Churned = RFM_Customer_Alive_VIP[(RFM_Customer_Alive_VIP['Churn'] >0.5) & (RFM_Customer_Alive_VIP['Churn'] <= 1)].count()/RFM_Customer_Alive_VIP.count()
dataframe = {'Safe':[safe.customer_id],'Waving':[Waving.customer_id], 'Accelerated_Churn': [Accelerated_Churn.customer_id], 'Churned':[Churned.customer_id]}

vip_dataframe = pd.DataFrame(dataframe)
vip_dataframe.rename(index={0: 'VIP'}, inplace= True)
vip_dataframe

#### Loyal

In [ ]:
RFM_Customer_Alive_loyal = RFM_Customer_Alive[RFM_Customer_Alive['LOYALTY_GROUP_MD'] == 'Loyal']
safe = RFM_Customer_Alive_loyal[(RFM_Customer_Alive_loyal['Churn'] >=0) & (RFM_Customer_Alive_loyal['Churn'] <= 0.05)].count()/RFM_Customer_Alive_loyal.count()
Waving = RFM_Customer_Alive_loyal[(RFM_Customer_Alive_loyal['Churn'] >0.05) & (RFM_Customer_Alive_loyal['Churn'] <= 0.2)].count()/RFM_Customer_Alive_loyal.count()
Accelerated_Churn = RFM_Customer_Alive_loyal[(RFM_Customer_Alive_loyal['Churn'] >0.2) & (RFM_Customer_Alive_loyal['Churn'] <= 0.5)].count()/RFM_Customer_Alive_loyal.count()
Churned = RFM_Customer_Alive_loyal[(RFM_Customer_Alive_loyal['Churn'] >0.5) & (RFM_Customer_Alive_loyal['Churn'] <= 1)].count()/RFM_Customer_Alive_loyal.count()
dataframe = {'Safe':[safe.customer_id],'Waving':[Waving.customer_id], 'Accelerated_Churn': [Accelerated_Churn.customer_id], 'Churned':[Churned.customer_id]}

loyal_dataframe = pd.DataFrame(dataframe)
loyal_dataframe.rename(index={0: 'LOYAL'}, inplace= True)
loyal_dataframe

#### Habitual

In [ ]:
RFM_Customer_Alive_Habitual = RFM_Customer_Alive[RFM_Customer_Alive['LOYALTY_GROUP_MD'] == 'Habitual']
safe = RFM_Customer_Alive_Habitual[(RFM_Customer_Alive_Habitual['Churn'] >=0) & (RFM_Customer_Alive_Habitual['Churn'] <= 0.05)].count()/RFM_Customer_Alive_Habitual.count()
Waving = RFM_Customer_Alive_Habitual[(RFM_Customer_Alive_Habitual['Churn'] >0.05) & (RFM_Customer_Alive_Habitual['Churn'] <= 0.2)].count()/RFM_Customer_Alive_Habitual.count()
Accelerated_Churn = RFM_Customer_Alive_Habitual[(RFM_Customer_Alive_Habitual['Churn'] >0.2) & (RFM_Customer_Alive_Habitual['Churn'] <= 0.5)].count()/RFM_Customer_Alive_Habitual.count()
Churned = RFM_Customer_Alive_Habitual[(RFM_Customer_Alive_Habitual['Churn'] >0.5) & (RFM_Customer_Alive_Habitual['Churn'] <= 1)].count()/RFM_Customer_Alive_Habitual.count()
dataframe = {'Safe':[safe.customer_id],'Waving':[Waving.customer_id], 'Accelerated_Churn': [Accelerated_Churn.customer_id], 'Churned':[Churned.customer_id]}

Habitual_dataframe = pd.DataFrame(dataframe)
Habitual_dataframe.rename(index={0: 'HABITUAL'}, inplace= True)
Habitual_dataframe

#### Desirables

In [ ]:
RFM_Customer_Alive_Desirables = RFM_Customer_Alive[RFM_Customer_Alive['LOYALTY_GROUP_MD'] == 'Desirables']
safe = RFM_Customer_Alive_Desirables[(RFM_Customer_Alive_Desirables['Churn'] >=0) & (RFM_Customer_Alive_Desirables['Churn'] <= 0.05)].count()/RFM_Customer_Alive_Desirables.count()
Waving = RFM_Customer_Alive_Desirables[(RFM_Customer_Alive_Desirables['Churn'] >0.05) & (RFM_Customer_Alive_Desirables['Churn'] <= 0.2)].count()/RFM_Customer_Alive_Desirables.count()
Accelerated_Churn = RFM_Customer_Alive_Desirables[(RFM_Customer_Alive_Desirables['Churn'] >0.2) & (RFM_Customer_Alive_Desirables['Churn'] <= 0.5)].count()/RFM_Customer_Alive_Desirables.count()
Churned = RFM_Customer_Alive_Desirables[(RFM_Customer_Alive_Desirables['Churn'] >0.5) & (RFM_Customer_Alive_Desirables['Churn'] <= 1)].count()/RFM_Customer_Alive_Desirables.count()
dataframe = {'Safe':[safe.customer_id],'Waving':[Waving.customer_id], 'Accelerated_Churn': [Accelerated_Churn.customer_id], 'Churned':[Churned.customer_id]}

Desirables_dataframe = pd.DataFrame(dataframe)
Desirables_dataframe.rename(index={0: 'DESIRABLES'}, inplace= True)
Desirables_dataframe

#### Others

In [ ]:
RFM_Customer_Alive_Others = RFM_Customer_Alive[RFM_Customer_Alive['LOYALTY_GROUP_MD'].isin(['Irregular','Switchers','Casual / Top-Ups','Unassigned	'])]
safe = RFM_Customer_Alive_Others[(RFM_Customer_Alive_Others['Churn'] >=0) & (RFM_Customer_Alive_Others['Churn'] <= 0.05)].count()/RFM_Customer_Alive_Others.count()
Waving = RFM_Customer_Alive_Others[(RFM_Customer_Alive_Others['Churn'] >0.05) & (RFM_Customer_Alive_Others['Churn'] <= 0.2)].count()/RFM_Customer_Alive_Others.count()
Accelerated_Churn = RFM_Customer_Alive_Others[(RFM_Customer_Alive_Others['Churn'] >0.2) & (RFM_Customer_Alive_Others['Churn'] <= 0.5)].count()/RFM_Customer_Alive_Others.count()
Churned = RFM_Customer_Alive_Others[(RFM_Customer_Alive_Others['Churn'] >0.5) & (RFM_Customer_Alive_Others['Churn'] <= 1)].count()/RFM_Customer_Alive_Others.count()
dataframe = {'Safe':[safe.customer_id],'Waving':[Waving.customer_id], 'Accelerated_Churn': [Accelerated_Churn.customer_id], 'Churned':[Churned.customer_id]}

Others_dataframe = pd.DataFrame(dataframe)
Others_dataframe.rename(index={0: 'OTHERS'}, inplace= True)
Others_dataframe

In [ ]:
%pylab inline
import pandas as pd
import matplotlib.pyplot as plt

ax = result.plot(kind='bar', stacked=True)
plt.ylabel('Portion of Customers by Each Segment')
plt.title('Probability of Churned by Loyalty Group')

### 2.3 Plotting the probability of churn graphs

In [ ]:
from lifetimes.utils import calculate_alive_path
def plot_history_alive(model, t, transactions, datetime_col, freq="D", start_date=None, ax=None, **kwargs):
    """
    Draw a graph showing the probability of being alive for a customer in time.
    Parameters
    ----------
    model: lifetimes model
        A fitted lifetimes model.
    t: int
        the number of time units since the birth we want to draw the p_alive
    transactions: pandas DataFrame
        DataFrame containing the transactions history of the customer_id
    datetime_col: str
        The column in the transactions that denotes the datetime the purchase was made
    freq: str, optional
        Default 'D' for days. Other examples= 'W' for weekly
    start_date: datetime, optional
        Limit xaxis to start date
    ax: matplotlib.AxesSubplot, optional
        Using user axes
    kwargs
        Passed into the matplotlib.pyplot.plot command.
    Returns
    -------
    axes: matplotlib.AxesSubplot
    """
    from matplotlib import pyplot as plt

    if start_date is None:
        start_date = min(transactions[datetime_col])

    if ax is None:
        ax = plt.subplot(111)

    # Get purchasing history of user
    customer_history = transactions[[datetime_col]].copy()
    customer_history.index = pd.DatetimeIndex(customer_history[datetime_col])

    # Add transactions column
    customer_history["transactions"] = 1
    customer_history = customer_history.resample(freq).sum()

    # plot alive_path
    path = calculate_alive_path(model, transactions, datetime_col, t, freq)
    path = 1 - path
    path_dates = pd.date_range(start=min(transactions[datetime_col]), periods=len(path), freq=freq)
    plt.plot(path_dates, path, "-", label="Probability of Churn")

    # plot buying dates
    payment_dates = customer_history[customer_history["transactions"] >= 1].index
    plt.vlines(payment_dates.values, ymin=0, ymax=1, colors="r", linestyles="dashed", label="daily purchases")

    plt.ylim(0, 1.0)
    plt.yticks(np.arange(0, 1.1, 0.1))
    plt.xlim(start_date, path_dates[-1])
    plt.legend(loc=3)
    plt.ylabel("Probability of Churn")
    plt.title("History of Probability of Churn")

    return ax

import matplotlib.pyplot as plt
data['date'] = pd.to_datetime(data.date, format="%Y%m%d")
data_test = data[data['date'] >= '2018-03-01']
fig = plt.figure(figsize=(12,8))
id = 2100000002
days_since_birth = 300
sp_trans = data_test.loc[data_test['customer_id'] == id]
plot_history_alive(bgf, days_since_birth, sp_trans, 'date')

### 2.4 Customer Probability of Churn Path

In [ ]:
# calculate_alive_path function can calculate the probability of churn for every shopping day for one customer
# For example the customer 2101823123, the above graph also shows this custoemr probability of churn changing
from lifetimes.utils import calculate_alive_path

data['date'] = pd.to_datetime(data.date, format="%Y%m%d")
data_test = data[data['date'] >= '2017-10-01']
id = 2101275685
days_since_birth = 300
sp_trans = data_test.loc[data_test['customer_id'] == id]
alive_path = pd.DataFrame(1-calculate_alive_path(bgf, sp_trans, 'date', days_since_birth, freq='D'))
alive_path.columns = ['Churn']
alive_path.head(10)

###  2.5 Prediction Date Range

In [ ]:
# Importing the data
chunksize = 100000
churn = pd.read_csv('data_20171001_20190930.csv', chunksize=100000, iterator=True)
data = pd.concat(churn, ignore_index=True)

# Manipulate the day range
date_range = data[(data['date'] >= 20181001) &(data['date'] <= 20190930)]
date_range['date'] = pd.to_datetime(date_range.date, format="%Y%m%d")

min=pd.DataFrame(date_range.groupby('customer_id')['date'].min())
max=pd.DataFrame(date_range.groupby('customer_id')['date'].max())
min_max = pd.concat([min,max], axis=1)
min_max['Date_Range'] = min_max.iloc[:,1] - min_max.iloc[:,0]

min_max.columns = ['min_date','max_date','date_range']
min_max['date_range'] = min_max['date_range'].dt.days
min_max.head()

# 3. Predicting the customer average daily spend

### 3.1 Delte Outliers for Training Dataset

In [ ]:
# Importing data
# Import the Training and test dataset
training = pd.read_csv("training_day.csv",index_col='customer_id')
test = pd.read_csv("test_day.csv",index_col='customer_id')

#Delete the outlier
lower_bound = 0.1
upper_bound = 0.95
training.frequency.quantile([lower_bound,upper_bound])

In [ ]:
# Looking the rfm frequency distribution
%matplotlib inline
import matplotlib.pyplot as plt
training['frequency'].plot(kind='hist', bins=50)
plt.ylabel('Number of Customers')
plt.xlabel('Frequency')
plt.title('Customer Frequency Distribution')

In [ ]:
# Looking the rfm frequency distribution
import matplotlib.pyplot as plt
plt.xlim([0,1000])
training['monetary_value'].plot(kind='hist', bins=1000)
plt.ylabel('Number of Customers')
plt.xlabel('Monetary value')
plt.title('Customer Monetary value Distribution')

In [ ]:
# Looking more detail about the monetary value
import matplotlib.pyplot as plt
plt.xlim([250,1000])
plt.ylim([0,2100])
training['monetary_value'].plot(kind='hist', bins=1000)

In [ ]:
# delete the outlier
training = training.loc[(training['frequency'] > 0) & (training['frequency'] < 250)& (training['monetary_value'] <= 400)]

In [ ]:
training[['frequency', 'monetary_value']].corr()

In [ ]:
corr = {'Repeated Daily Purchase':[1.000000,0.014694], 'Average Daily Spend': [1.000000,0.014694]}
corr = pd.DataFrame(corr)
corr.rename(index = {0:'Repeated Daily Purchase',1:'Average Daily Spend'})

### 3.2 Building Model

In [ ]:
# refit the BG model to the summary_with_money_value dataset
from lifetimes import BetaGeoFitter
bgf = BetaGeoFitter(penalizer_coef=0.08)
bgf.fit(training['frequency'], training['recency'], training['T'])
bgf.summary

In [ ]:
# Build the model
training = training[training['frequency']>0]
training = training[training['monetary_value']>0]

from lifetimes import GammaGammaFitter
ggf = GammaGammaFitter(penalizer_coef = 0.01)
ggf.fit(training['frequency'],
        training['monetary_value'])
print(ggf.summary)

In [ ]:
print("Expected conditional average profit: %s, Average profit: %s" % (
    ggf.conditional_expected_average_profit(
        training['frequency'],
        training['monetary_value']
    ).mean(),
    training[training['frequency']>0]['monetary_value'].mean()
))

In [ ]:
training_predict = ggf.conditional_expected_average_profit(
        training['frequency'],
        training['monetary_value'])

training_actual = training['monetary_value']
training_predict_actual = pd.concat([training_actual,training_predict],axis =1)
# Rename the column name
training_predict_actual.columns = ['Actual', 'Predict']
training_predict_actual['difference'] = (training_predict_actual['Predict']-training_predict_actual['Actual'])/training_predict_actual['Actual']*100
training_predict_actual.head()

In [ ]:
#All cusotmer 
import matplotlib.pyplot as plt
%matplotlib inline
plt.xlim([0,630])
plt.ylim([0,630])
x = training_predict_actual['Actual'] 
y = training_predict_actual['Predict']

plt.plot(x,y,'r.') # x vs y
plt.plot(x,x,'k-') # identity line

plt.ylabel('Prediction Monetary Value')
plt.xlabel('Actual Monetary Value')
plt.title('Expected conditional average spend for Calibration Period')

plt.scatter(x, y, alpha=0.5)
plt.show()

In [ ]:
prediction_training = training_predict_actual['Predict'].sum()
actual_training = training_predict_actual['Actual'].sum()
print("Total Amount Prediction:", prediction_training)
print("Total Amount holdout:", actual_training)
print("Difference:", prediction_training - actual_training)
print("Prediction Error :",(prediction_training-actual_training)/actual_training)

### 3.3 Delte Outliers for Test Dataset

In [ ]:
#Delete the outlier
lower_bound = 0.1
upper_bound = 0.95
test.frequency.quantile([lower_bound,upper_bound])

In [ ]:
# Looking the rfm frequency distribution
%matplotlib inline
test['frequency'].plot(kind='hist', bins=50)

In [ ]:
# Looking the rfm frequency distribution
import matplotlib.pyplot as plt
plt.xlim([0,1000])
test['monetary_value'].plot(kind='hist', bins=1000)

In [ ]:
# More detail about monetary value
import matplotlib.pyplot as plt
plt.xlim([250,1000])
plt.ylim([0,2100])
test['monetary_value'].plot(kind='hist', bins=1000)

In [ ]:
# Not delete the outlier and have a try
test = test.loc[(test['frequency'] > 0) & (test['frequency'] < 250)& (test['monetary_value'] <= 500)]

In [ ]:
test = test.loc[(test['frequency'] > 0) & (test['monetary_value'] > 0)]

print("Expected conditional average profit: %s, Average profit: %s" % (
    ggf.conditional_expected_average_profit(
        test['frequency'],
        test['monetary_value']
    ).mean(),
    test[test['frequency']>0]['monetary_value'].mean()
))

In [ ]:
test_predict = ggf.conditional_expected_average_profit(
        test['frequency'],
        test['monetary_value'])
test_actual = test['monetary_value']
test_predict_actual = pd.concat([test_actual,test_predict],axis =1)
# Rename the column name
test_predict_actual.columns = ['Actual', 'Predict']
test_predict_actual['difference'] = (test_predict_actual['Predict']-test_predict_actual['Actual'])/test_predict_actual['Actual']
test_predict_actual.head()

In [ ]:
#All cusotmer 
import matplotlib.pyplot as plt
%matplotlib inline
plt.xlim([0,620])
plt.ylim([0,620])
x = test_predict_actual['Actual'] 
y = test_predict_actual['Predict']

plt.plot(x,y,'r.') # x vs y
plt.plot(x,x,'k-') # identity line

plt.ylabel('Prediction Monetary Value')
plt.xlabel('Actual Monetary Value')
plt.title('Expected conditional average spend for Holdout Period')

plt.scatter(x, y, alpha=0.5)
plt.show()

In [ ]:
prediction_test = test_predict_actual['Predict'].sum()
actual_test = test_predict_actual['Actual'].sum()
print("Total Amount Prediction:", prediction_test)
print("Total Amount holdout:", actual_test)
print("Difference:", prediction_test - actual_test)
print("Prediction Error :",(prediction_test-actual_test)/actual_test)

# 4. Predicting the customer total monetary value in next 1 year

### 4.1 Model

In [ ]:
# Reading the dataset
training = pd.read_csv("training_day.csv",index_col='customer_id')
test = pd.read_csv("test_day.csv",index_col='customer_id')


# refit the BG model to the summary_with_money_value dataset
from lifetimes import BetaGeoFitter
bgf = BetaGeoFitter(penalizer_coef=0.05)
training_model1 = training.loc[(training['frequency'] > 0) & (training['frequency'] < 250)]
bgf.fit(training_model1['frequency'], training_model1['recency'], training_model1['T'])

training_model2 = training.loc[(training['frequency'] > 0) & (training['frequency'] < 250)& (training['monetary_value'] <= 400)]
from lifetimes import GammaGammaFitter
ggf = GammaGammaFitter(penalizer_coef = 0.05)
ggf.fit(training_model2['frequency'],
        training_model2['monetary_value'])

training = training[(training['frequency'] > 0) & (training['monetary_value'] > 0)]
customer_lifetime_value = ggf.customer_lifetime_value(
    bgf, #the model to use to predict the number of future transactions
    training['frequency'],
    training['recency'],
    training['T'],
    training['monetary_value'],
    time=12, # months
    discount_rate=0, # Yearly discount rate ~6.0%  p.a  = 0.5% per month  
    freq = 'D'
)

### 4.2 Evaluation

In [ ]:
data_20181001_20190930 = data[(data['date'] >=20181001) & (data['date'] <=20190930)]

Monetary_Value_Predict_Actual = pd.concat([customer_lifetime_value, data_20181001_20190930.groupby(['customer_id']).sum()['amount']], axis = 1)
Monetary_Value_Predict_Actual.columns = ['Predict', 'Actual']
Monetary_Value_Predict_Actual = Monetary_Value_Predict_Actual.dropna()
prediction = Monetary_Value_Predict_Actual['Predict'].sum()
actual = Monetary_Value_Predict_Actual['Actual'].sum()
print("Total Amount Prediction:", prediction)
print("Total Amount holdout:", actual)
print("Difference:", prediction - actual)
print("Prediction Error :",(prediction-actual)/actual)

In [ ]:
#All cusotmer 
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline


ax = Monetary_Value_Predict_Actual.plot(kind='scatter', x='Actual', y='Predict')

formatter = ticker.FormatStrFormatter('$%0.1f') #declaring the formatter with the $ sign and y_values with 1 decimalplace
ax.yaxis.set_major_formatter(formatter)
ax.xaxis.set_major_formatter(formatter)

x = Monetary_Value_Predict_Actual['Actual'] 
y = Monetary_Value_Predict_Actual['Predict']
plt.plot(x,y,'r.') # x vs y
plt.plot(x,x,'k-') # identity line

plt.xlim([0,100000])
plt.ylim([0,100000])

plt.ylabel('Prediction Monetary Value')
plt.xlabel('Actual Monetary Value')
plt.title('Actual Montary Value VS Prediction Monetary Value')

mpl.rcParams['agg.path.chunksize'] = 100000
plt.scatter(x, y, alpha=0.5)
plt.show()

In [ ]:
# Get the difference for every customer
Monetary_Value_Predict_Actual['Difference'] = round((Monetary_Value_Predict_Actual['Predict'] - Monetary_Value_Predict_Actual['Actual'])/Monetary_Value_Predict_Actual['Actual'],0)
difference = pd.DataFrame(Monetary_Value_Predict_Actual["Difference"].value_counts())
difference = difference.sort_index(ascending=True)

difference_ten = difference.loc[-1:10]
difference_ten = difference_ten.append({'Difference' : 28832 } , ignore_index=True)
difference_ten.rename(index={0: -1, 1:0, 2: 1, 3:2, 4: 3, 5:4, 6: 5, 7:6, 8: 7, 9:8, 10: 9, 11: 10,12: '10+'}, inplace= True)

ax = difference_ten.plot(kind='bar')
plt.xlim([-1,13])
plt.ylabel('Number of customers')
plt.xlabel('Prediction Range')
plt.title('Prediction Range Distribution')
ax.legend().set_visible(False)
plt.show()

In [ ]:
# Get the difference for every customer
Monetary_Value_Predict_Actual['Difference'] = round((Monetary_Value_Predict_Actual['Predict'] - Monetary_Value_Predict_Actual['Actual'])/Monetary_Value_Predict_Actual['Actual'],1)
difference = pd.DataFrame(Monetary_Value_Predict_Actual["Difference"].value_counts())
difference = difference.sort_index(ascending=True)
difference_range = difference[(difference['Difference']>=-1) & (difference['Difference']<=1)]

ax = difference.plot(kind='line')
plt.xlim([-1,1])
plt.ylabel('Number of customers')
plt.xlabel('Prediction Range')
plt.title('Prediction Range Distribution from -1 to 1')
ax.legend().set_visible(False)
plt.show()

### 4.3 Visullization for next 5 years

In [ ]:
# Reading the dataset
training = pd.read_csv("training_day.csv",index_col='customer_id')
test = pd.read_csv("test_day.csv",index_col='customer_id')


# refit the BG model to the summary_with_money_value dataset
from lifetimes import BetaGeoFitter
bgf = BetaGeoFitter(penalizer_coef=0.05)
training_model1 = training.loc[(training['frequency'] > 0) & (training['frequency'] < 250)]
bgf.fit(training_model1['frequency'], training_model1['recency'], training_model1['T'])

training_model2 = training.loc[(training['frequency'] > 0) & (training['frequency'] < 250)& (training['monetary_value'] <= 400)]
from lifetimes import GammaGammaFitter
ggf = GammaGammaFitter(penalizer_coef = 0.05)
ggf.fit(training_model2['frequency'],
        training_model2['monetary_value'])

training = training[(training['frequency'] > 0) & (training['monetary_value'] > 0)]
customer_lifetime_value_2019 = ggf.customer_lifetime_value(
    bgf, #the model to use to predict the number of future transactions
    training['frequency'],
    training['recency'],
    training['T'],
    training['monetary_value'],
    time=12, # months
    discount_rate=0.005, # Yearly discount rate ~6.0%  p.a  = 0.5% per month  
    freq = 'D'
)

In [ ]:
# Reading the dataset
training = pd.read_csv("training_day.csv",index_col='customer_id')
test = pd.read_csv("test_day.csv",index_col='customer_id')


# refit the BG model to the summary_with_money_value dataset
from lifetimes import BetaGeoFitter
bgf = BetaGeoFitter(penalizer_coef=0.05)
training_model1 = training.loc[(training['frequency'] > 0) & (training['frequency'] < 250)]
bgf.fit(training_model1['frequency'], training_model1['recency'], training_model1['T'])

training_model2 = training.loc[(training['frequency'] > 0) & (training['frequency'] < 250)& (training['monetary_value'] <= 400)]
from lifetimes import GammaGammaFitter
ggf = GammaGammaFitter(penalizer_coef = 0.05)
ggf.fit(training_model2['frequency'],
        training_model2['monetary_value'])

training = training[(training['frequency'] > 0) & (training['monetary_value'] > 0)]
customer_lifetime_value_2024 = ggf.customer_lifetime_value(
    bgf, #the model to use to predict the number of future transactions
    training['frequency'],
    training['recency'],
    training['T'],
    training['monetary_value'],
    time=72, # months
    discount_rate=0.005, # Yearly discount rate ~6.0%  p.a  = 0.5% per month  
    freq = 'D'
)

In [ ]:
# total clv 2024 minus total clv 2019
customer_lifetime_value_5years = customer_lifetime_value_2024-customer_lifetime_value_2019
customer_lifetime_value_5years.head()

In [ ]:
# Plotting the distribution
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
plt.xlim([0,100000])
plt.ylim([0,400000])

ax = customer_lifetime_value_5years.plot(kind='hist', bins=1000)

formatter = ticker.FormatStrFormatter('$%0.1f') #declaring the formatter with the $ sign and y_values with 1 decimalplace
ax.xaxis.set_major_formatter(formatter)

plt.ylabel('Number of Customers')
plt.xlabel('Total $dollar Spend from 20191001 to 20240930')
plt.title('Customers total $dollar Spend for next 5 years distrbution (6% Discount Rate)')

In [ ]:
# Merge the probability of churn and CLV next 5 years
clv_5years = pd.concat([customer_lifetime_value_5years,test_probability_alive['Churn']], axis = 1).dropna()

x = clv_5years['Churn']
y = clv_5years['clv']
ax = plt.scatter(x, y, alpha=0.5)

plt.ylim([0,750000])
plt.ylabel('Total $Spend from 2020 to 2024')
plt.xlabel('Probability of Churn')
plt.title('Customer next  5 years lifetime value with probability of churn')
#ax.legend().set_visible(False)
plt.show()

In [ ]:
# For the Waving customers
clv_5years_waving = clv_5years[(clv_5years['Churn'] >0.05) & (clv_5years['Churn'] <0.2)]

colors = np.where(clv_5years_waving["clv"]>=50000,'r','b')
ax = clv_5years_waving.plot(kind='scatter', x='Churn', y='clv',c=colors)

formatter = ticker.FormatStrFormatter('$%0.1f') #declaring the formatter with the $ sign and y_values with 1 decimalplace
ax.yaxis.set_major_formatter(formatter)

#plt.ylim([0,750000])
plt.ylabel('Total $Spend from 2020 to 2024')
plt.xlabel('Probability of Waving customer')
plt.title('Waving customer next  5 years lifetime value with probability of churn')
#ax.legend().set_visible(False)
plt.show()

In [ ]:
clv_5years_waving[clv_5years_waving["clv"]>=50000].shape

In [ ]:
#No Discount 
a= {'Actual':[4341439234,4287138878, None, None, None, None,None],'Prediction':[None,4289766150,4247675722,4221221544,4201801372,4186447729,4173752048] }
a_dataframe = pd.DataFrame(a)
a_dataframe
a_dataframe.rename(index={0: 2018, 1:2019, 2: 'Next1Year', 3:'Next2Year', 4: 'Next3Year', 5:'Next4Year', 6: 'Next5Year'}, inplace= True)

%matplotlib inline
import matplotlib.pyplot as plt

ax = a_dataframe.plot(kind='line',marker='o')

plt.ylabel('Totale sale (Unit: Billion)')
plt.xlabel('Year')
plt.title('Customer Lifetime Value for Next 5 Years')
#ax.legend().set_visible(False)
plt.show()

In [ ]:
# With Discount = 0.005%
a= {'Actual':[4341439234,4287138878, None, None, None, None,None],'Prediction':[None,4153813107,3873999987,3626176354,3399781349,3190559058,2996083484] }
a_dataframe = pd.DataFrame(a)
a_dataframe
a_dataframe.rename(index={0: 2018, 1:2019, 2: 'Next1Year', 3:'Next2Year', 4: 'Next3Year', 5:'Next4Year', 6: 'Next5Year'}, inplace= True)

%matplotlib inline
import matplotlib.pyplot as plt

ax = a_dataframe.plot(kind='line',marker='o')

plt.ylabel('Totale sale (Unit: Billion)')
plt.xlabel('Year')
plt.title('Customer Lifetime Value for Next 5 Years with 6% Discount Rate')
#ax.legend().set_visible(False)
plt.show()